**<h3><center>EN2550 Exercise 7 - Camera and Calibration</center></h3>**
*Name - Abeysinghe W.A.M.S.Y*
<br>
*Index no - 190014F*
<br>

All the imports are done at the beginning. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

**Hough Transforms**

**Q1**. Fit Houghlines to the Soduko image in Fing. 1.

**Q2**. Fit Hough circles to the coins image in Fing. 3.

**Q3**. Use generalized Hough transform to match the template with the image.

**Alignment**

**Q4**. Using he given code (item no. 1), experiment with various types of 2-D transformations.

**Q5**. Transform Graffiti img1.ppm onto img5.ppm using code in item no. 2.